# Week 6 Coding Exercises

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from utils import display_image, display_images

1. Apply custom sharpening kernel of aperture size 3 and 5 as shown below on 'native-bee.png':  
$ 3 \times 3$ kernel:  
$ \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0\\
\end{bmatrix}$  
$ 5 \times 5$ kernel:  
$ \begin{bmatrix}
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & 25 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
\end{bmatrix}$  
What can you infer from the outputs?

In [2]:
kernel_3x3 = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1],
                       [-1, -1, 25, -1, -1],
                       [-1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1]])

img = cv.imread("images/native-bee.png")

dst_3x3 = cv.filter2D(img, -1, kernel_3x3)
dst_5x5 = cv.filter2D(img, -1, kernel_5x5)

display_images([img, dst_3x3, dst_5x5], ("source", "3 x 3 kernel", "5 x 5 kernel"))


# The 3x3 sharpened image show fine details more crisply, while the 5x5 sharpened image show stronger edges and more noise.
# 3x3 sharpening kernel emphasizes the center pixel by subtracting the neighboring pixels' values from it
#, enhancing the edges and details in the image.
# 5x5 sharpening kernel considers more neighboring pixels 
# , provides a more pronounced sharpening effect, capturing larger details and more extensive edges.

2. Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.

In [3]:
img = cv.imread("images/noise_lena.jpg")

# average filter -> smooth out the image uniformly but blur the edges and details
avg_filter = cv.blur(img, (5, 5))

# Gaussian filter -> similar to the average filter but gives more weight to the central pixels
gaussian_filter = cv.GaussianBlur(img, (5, 5), 0)
    
# median filter -> it replaces each pixel value with the median of its neighbors, preserving edges while removing noise.
median_filter = cv.medianBlur(img, 5)

display_images([img, avg_filter, gaussian_filter, median_filter], ["Original", "Average Filter", "Gaussian Filter", "Median Filter"])

# From the outcomes, type of noise can be deduced is Gauusian Noise 
# if average and Gaussian filters perform well in reducing the noise, 
# wheareas Salt-and-Pepper Noise if median filter performs better in removing noise compared to the other filters.

3. Write a program to *segment the boat and the people on it from the background*. Follow the instruction below:
    - Use 'boat.jpg' as input.
    - Apply Otsu thresholding.
    - Draw bounding box to identify the region where the boat and people are located.

In [15]:
img = cv.imread("images/boat.jpg")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# display_image("gray", gray)

# thresholding
# blur = cv.GaussianBlur(gray,(5, 5), 0)
th = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
# display_image("threshold", th)
 
# find contours
contours = cv.findContours(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
len(contours) # 14

# get the largest contour (post processing)
contour_largest = sorted(contours, key=cv.contourArea, reverse=True)[0]
    
# draw contours
img_copy = img.copy()
cv.drawContours(img_copy, [contour_largest], 0, (0, 0, 255), 2)
# display_image("contour", img_copy)

# draw bounding box
img_copy = img.copy()
x,y,w,h = cv.boundingRect(contour_largest)
cv.rectangle(img_copy, (x,y), (x+w, y+h), (0, 255, 0), 2)
display_image("boat and the people on it", img_copy)